In [1]:
!pip install -q kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle

In [4]:
!kaggle datasets download -d trainingdatapro/knee-x-rays-4-stages-of-osteoarthritis

 99% 391M/395M [00:12<00:00, 37.2MB/s]
100% 395M/395M [00:12<00:00, 32.9MB/s]


In [5]:
!unzip /content/knee-x-rays-4-stages-of-osteoarthritis.zip

Archive:  /content/knee-x-rays-4-stages-of-osteoarthritis.zip
  inflating: files/arthritis/0.dcm   
  inflating: files/arthritis/0.jpg   
  inflating: files/arthritis/1.dcm   
  inflating: files/arthritis/1.jpg   
  inflating: files/arthritis/2.dcm   
  inflating: files/arthritis/2.jpg   
  inflating: files/arthritis/3.dcm   
  inflating: files/arthritis/3.jpg   
  inflating: files/arthritis/4.dcm   
  inflating: files/arthritis/4.jpg   
  inflating: files/arthritis/5.dcm   
  inflating: files/arthritis/5.jpg   
  inflating: files/arthritis/6.dcm   
  inflating: files/arthritis/6.jpg   
  inflating: files/arthritis/7.dcm   
  inflating: files/arthritis/7.jpg   
  inflating: files/arthritis/8.dcm   
  inflating: files/arthritis/8.jpg   
  inflating: files/fracture/0.dcm    
  inflating: files/fracture/0.jpg    
  inflating: files/fracture/1.dcm    
  inflating: files/fracture/1.jpg    
  inflating: files/fracture/2.dcm    
  inflating: files/fracture/2.jpg    
  inflating: files/fractur

In [6]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [8]:
!pip install Augmentor

In [9]:
import Augmentor

p = Augmentor.Pipeline(r"/content/files")

p.zoom(probability=0.3, min_factor=0.8, max_factor=1.5)
p.flip_top_bottom(probability=0.4)
p.random_brightness (probability=0.3,min_factor=0.3,max_factor=1.2)
p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=8)
p.sample(500)

Initialised with 50 image(s) found.
Output directory set to /content/files/output.

Processing <PIL.Image.Image image mode=RGB size=3072x2560 at 0x7EF5D2BF7040>: 100%|██████████| 500/500 [05:39<00:00,  1.47 Samples/s]


In [10]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the path to your original image dataset
dataset_path = '/content/files/output'

# Define the paths for the train, test, and valid folders
train_path = '/content/train'
test_path = '/content/test'
#valid_path = '/content/valid'

# Create the train, test, and valid folders if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)
#os.makedirs(valid_path, exist_ok=True)

# Get the list of classes (subdirectories) in the dataset
classes = os.listdir(dataset_path)

# Iterate over each class
for class_name in classes:
    class_path = os.path.join(dataset_path, class_name)
    class_images = [os.path.join(class_path, img) for img in os.listdir(class_path)]

    # Split the images into training, testing, and validation sets
    train_images, test_images = train_test_split(class_images, test_size=0.3, random_state=42)
    #valid_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Define the paths for the train, test, and valid sets for the current class
    train_class_path = os.path.join(train_path, class_name)
    test_class_path = os.path.join(test_path, class_name)
    #valid_class_path = os.path.join(valid_path, class_name)

    # Create the train, test, and valid class folders if they don't exist
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(test_class_path, exist_ok=True)
    #os.makedirs(valid_class_path, exist_ok=True)

    # Copy images to their respective folders
    for img_path in train_images:
        shutil.copy(img_path, os.path.join(train_class_path, os.path.basename(img_path)))

    for img_path in test_images:
        shutil.copy(img_path, os.path.join(test_class_path, os.path.basename(img_path)))

    #for img_path in valid_images:
        #shutil.copy(img_path, os.path.join(valid_class_path, os.path.basename(img_path)))

In [11]:
trainpath = '/content/train'
testpath = '/content/test'

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,zoom_range= 0.2,shear_range= 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
train = train_datagen.flow_from_directory(trainpath,target_size =(224,224),batch_size = 20)
test = test_datagen.flow_from_directory(testpath,target_size =(224,224),batch_size = 20) ,#5 ,15 , 32, 50

Found 348 images belonging to 6 classes.
Found 152 images belonging to 6 classes.


## VGG 16

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

In [15]:
vgg = VGG16(include_top = False,input_shape = (224,224,3))

58889256/58889256 [==============================] - 2s 0us/step


In [16]:
for layer in vgg.layers:
  print(layer)

In [17]:
len(vgg.layers)

19

In [18]:
for layer in vgg.layers:
  layer.trainable = False

In [19]:
x= Flatten()(vgg.output)

In [20]:
output = Dense(6, activation ='softmax')(x)

In [21]:
vgg16 = Model(vgg.input,output)

In [22]:
vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [23]:
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
opt = Adam(lr=0.0001)

# Assuming you have defined your VGG16 model as vgg16

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Compile the model (you may have already done this)
vgg16.compile(optimizer=opt , loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with early stopping callback
history = vgg16.fit(train, validation_data=test, epochs=50, steps_per_epoch=len(train), validation_steps=len(test), callbacks=[early_stopping])

Epoch 1/50
18/18 [==============================] - 29s 999ms/step - loss: 1.7572 - accuracy: 0.4195 - val_loss: 1.0095 - val_accuracy: 0.6000
Epoch 2/50
18/18 [==============================] - 15s 821ms/step - loss: 0.7122 - accuracy: 0.7730 - val_loss: 0.7174 - val_accuracy: 0.7000
Epoch 3/50
18/18 [==============================] - 15s 851ms/step - loss: 0.5040 - accuracy: 0.8247 - val_loss: 0.6179 - val_accuracy: 0.7500
Epoch 4/50
18/18 [==============================] - 16s 867ms/step - loss: 0.3267 - accuracy: 0.9023 - val_loss: 0.4817 - val_accuracy: 0.9000
Epoch 5/50
18/18 [==============================] - 14s 796ms/step - loss: 0.2270 - accuracy: 0.9569 - val_loss: 0.3426 - val_accuracy: 0.9000
Epoch 6/50
18/18 [==============================] - 17s 956ms/step - loss: 0.1812 - accuracy: 0.9828 - val_loss: 0.3322 - val_accuracy: 0.9000
Epoch 7/50
18/18 [==============================] - 16s 868ms/step - loss: 0.1400 - accuracy: 0.9885 - val_loss: 0.1885 - val_accuracy: 1.0000

In [24]:
vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [25]:
len(train)

18

In [26]:
len(test)

1

# save file .h5

In [27]:
vgg16.save('cnn_vgg16_Knee.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Testing the Data

In [28]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [29]:
labels=['arthritis','fracture','stage_1','stage_2','stage_3','stage_4']

# Testing Class 1

In [33]:
img_path ='/content/test/arthritis/arthritis_original_0.jpg_ab6b4f93-ee40-4418-a016-e1e8e0aae3d4.jpg'

In [34]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds

1/1 [==============================] - 0s 18ms/step


array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.5455692e-34, 0.0000000e+00]], dtype=float32)

In [35]:
labels[np.argmax(preds)]

'arthritis'

# Testing Class 2

In [36]:
img_path ='/content/test/fracture/fracture_original_1.jpg_e32721de-d588-470e-97b1-7b12c3eed687.jpg'

In [37]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds

1/1 [==============================] - 0s 19ms/step


array([[5.2963036e-26, 9.9999976e-01, 3.5583524e-24, 1.7941851e-07,
        3.2003110e-26, 0.0000000e+00]], dtype=float32)

In [38]:
labels[np.argmax(preds)]

'fracture'

# Testing Class 3

In [48]:
img_path ='/content/test/stage_1/stage_1_original_2.jpg_71f7afab-2d7c-4073-9dcc-a4e812804e2f.jpg'

In [49]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds

1/1 [==============================] - 0s 22ms/step


array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.8864311e-20,
        1.0037137e-32, 0.0000000e+00]], dtype=float32)

In [50]:
labels[np.argmax(preds)]

'stage_1'

# Testing Class 4

In [42]:
img_path ='/content/test/stage_2/stage_2_original_2.jpg_250e07c2-06ac-4e9b-84d3-b261eeea4685.jpg'

In [43]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds

1/1 [==============================] - 0s 18ms/step


array([[0.0000000e+00, 4.0287178e-37, 6.2783683e-27, 1.0000000e+00,
        2.7540141e-34, 0.0000000e+00]], dtype=float32)

In [44]:
labels[np.argmax(preds)]

'stage_2'

# Testing Class 5

In [60]:
img_path ='/content/test/stage_3/stage_3_original_7.jpg_e82fd6ca-dbd4-402e-94bd-7732614ae0e0.jpg'

In [61]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds

1/1 [==============================] - 0s 19ms/step


array([[0.0000000e+00, 1.5923744e-28, 2.5179585e-19, 3.9774910e-08,
        1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [62]:
labels[np.argmax(preds)]

'stage_3'

# Testing Class 6

In [69]:
img_path ='/content/test/stage_4/stage_4_original_1.jpg_efa31922-d933-4e49-a785-700caff77f10.jpg'

In [70]:
import numpy as np
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = preprocess_input(x)
preds = vgg16.predict(np.array([x]))
preds

1/1 [==============================] - 0s 29ms/step


array([[4.83866613e-10, 0.00000000e+00, 1.27486615e-11, 8.11765378e-19,
        3.62939209e-01, 6.37060761e-01]], dtype=float32)

In [71]:
labels[np.argmax(preds)]

'stage_4'